In [2]:
import cv2
import numpy as np

In [3]:
def region_of_interest(img, vertices):
    # Define a blank matrix that matches the image height/width.
    mask = np.zeros_like(img)
    
    #check if image is grayscale or color
    shape = img.shape
    if(len(shape) == 3):               #its a color image
        mask_color = (255,)*shape[-1]   #shape[-1] = no. at last index
    else:                              #otherwise its a gray image
        mask_color = 255
      
    # Fill the polygon with white
    cv2.fillPoly(mask, vertices, mask_color)
    
    # Returning the image only where mask pixels match
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

In [4]:
#Applying canny edge and thresholding using median of pixels
def autoCanny(img, sigma=0.33):
    '''
    img: A BGR image
    sigma: deviation from median color intensity
    '''
    #this produces gray image with better contrasting lane marks
    img = img[:,:,2] + img[:,:,1] - img[:,:,0]
    img = cv2.threshold(img, 0, 255, cv2.THRESH_TRUNC + cv2.THRESH_OTSU)
    cv2.imshow('gray', img)
    wt = True
    if wt:
        cv2.imwrite('gray1.jpg',img)
        wt = False
    #get median of gray scale image
    med = np.median(img)
    
    #compute lower and upper threshold
    lower = int(max(0, (1-sigma)*med))
    upper = int(min(255, (1+sigma)*med))
    edgedImg = cv2.Canny(img, 160, 255)
    
    #return image with prominent edges
    return edgedImg

In [5]:
cap = cv2.VideoCapture('../dataset/day-lane.mp4')

#get height and width of frame
ret, frame = cap.read()
height, width, channel = frame.shape

#keeping the portion of image which contains lane
topLeftPt = (width*(3.0/8), height*(3/5))
topRightPt = (width*(5.0/8), height*(3/5))

#polygon has 6 vertices
region_of_interest_points = [
(0, height),
(0, height*(3.4/5)),
topLeftPt,
topRightPt,
(width, height*(3.4/5)),
(width, height),
]

while True:

    # Take each frame
    ret, frame = cap.read()
    
    #crop image
    cropped = region_of_interest(frame, np.array([region_of_interest_points], np.int32))
    cv2.imshow('Region of Interest', cropped)
    
    #perform canny edge on gray scale image
    #cannyImg = autoCanny(cropped, 0.33)
    #cv2.imshow('cannyImg', cannyImg)
    
    k = cv2.waitKey(25) & 0xFF
    if k == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()